In [3]:
%%capture
!pip install datasets==1.13.3
!pip install transformers==4.11.3
!pip install jiwer

## Create Wav2Vec2CTCTokenizer

In [12]:
from datasets import load_dataset
import pandas as pd

In [9]:
all_data = load_dataset('csv',data_files='./order_speech_ko.csv',split='train')

Using custom data configuration default-41fc3c5f4c361674
Reusing dataset csv (C:\Users\Mu-jun\.cache\huggingface\datasets\csv\default-41fc3c5f4c361674\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


In [10]:
all_data

Dataset({
    features: ['src', 'text'],
    num_rows: 142367
})

In [14]:
df = pd.DataFrame(all_data)
df.sample()

,src,text
17316,script1_g_0071-6661-01-01-YJK-M-04-A.wav,내일 오존 주의보 떴어?


In [2]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [11]:
remove_spectial_char_data = all_data.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x000001824DC06DC8> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/142367 [00:00<?, ?ex/s]

In [13]:
df = pd.DataFrame(remove_spectial_char_data)
df.head()

,src,text
0,script1_g_0044-6001-01-01-KSM-F-05-A.wav,보고 있는 영상 정지시켜 줘
1,script1_g_0044-6002-01-01-KSM-F-05-A.wav,다음 주까지 날씨가 어때
2,script1_g_0044-6003-01-01-KSM-F-05-A.wav,나 대신 점등해 줘
3,script1_g_0044-6004-01-01-KSM-F-05-A.wav,이번 주 대체로 흐린지 궁금해
4,script1_g_0044-6005-01-01-KSM-F-05-A.wav,지금 당장 취침 등 꺼 줘


In [4]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [17]:
char_vocab = remove_spectial_char_data.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=remove_spectial_char_data.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
char_vocab

Dataset({
    features: ['all_text', 'vocab'],
    num_rows: 1
})

In [19]:
vocab_list = list(set(char_vocab["vocab"][0]))

In [20]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'캄': 0,
 '랜': 1,
 '딱': 2,
 '언': 3,
 '욱': 4,
 '슬': 5,
 '릴': 6,
 "'": 7,
 '뭔': 8,
 '뽑': 9,
 '라': 10,
 '택': 11,
 '십': 12,
 '즌': 13,
 '내': 14,
 '줄': 15,
 '왜': 16,
 'c': 17,
 '딸': 18,
 '밖': 19,
 '을': 20,
 '떼': 21,
 '곧': 22,
 '깜': 23,
 '컨': 24,
 '련': 25,
 'v': 26,
 '졌': 27,
 '거': 28,
 '꿀': 29,
 '둔': 30,
 '햇': 31,
 '읽': 32,
 '랐': 33,
 '정': 34,
 '아': 35,
 '끌': 36,
 '쉬': 37,
 '톱': 38,
 '럼': 39,
 '권': 40,
 '년': 41,
 '혹': 42,
 '이': 43,
 '캐': 44,
 '익': 45,
 '산': 46,
 '영': 47,
 'b': 48,
 '기': 49,
 '것': 50,
 '룸': 51,
 '품': 52,
 '흥': 53,
 '춤': 54,
 '트': 55,
 '매': 56,
 '맥': 57,
 '했': 58,
 '끄': 59,
 '루': 60,
 '켤': 61,
 '론': 62,
 '떨': 63,
 '랍': 64,
 '굽': 65,
 '디': 66,
 '흘': 67,
 '특': 68,
 '맙': 69,
 '축': 70,
 '키': 71,
 '쎄': 72,
 '류': 73,
 '균': 74,
 '둡': 75,
 '냄': 76,
 '미': 77,
 '포': 78,
 '화': 79,
 '엔': 80,
 '식': 81,
 '엘': 82,
 '야': 83,
 '말': 84,
 '친': 85,
 '짝': 86,
 '냐': 87,
 '괜': 88,
 '탕': 89,
 '색': 90,
 '입': 91,
 '게': 92,
 '띄': 93,
 '끝': 94,
 '관': 95,
 '별': 96,
 '꼭': 97,
 '풀': 98,
 '물': 99,
 '즉': 100,

In [21]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

602

In [22]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [23]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json",
                                 unk_token="[UNK]",
                                 pad_token="[PAD]",
                                 word_delimiter_token="|")

## Create XLSR-Wav2Vec2 Feature Extractor

In [24]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=True)

In [25]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor,
                              tokenizer=tokenizer)

In [26]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [27]:
# processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-ready")

## Preprocess Data

In [28]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

# Training

## Set-up Trainer

In [29]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

ModuleNotFoundError: No module named 'torch'

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
from datasets import load_dataset, load_metric, Audio

wer_metric = load_metric("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Import Model

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

XLSR-Wav2Vec2의 첫 번째 구성 요소는 원시 음성 신호에서 음향적으로 의미가 있지만 문맥적으로 독립적인 기능을 추출하는 데 사용되는 CNN 계층 스택으로 구성됩니다.  
모델의 이 부분은 사전 교육 중에 이미 충분히 훈련되었으며 논문에 명시된 바와 같이 더 이상 미세 조정할 필요가 없습니다. 따라서 특징 추출 부분의 모든 파라미터에 대해 require_grad를 False로 설정할 수 있다.

In [ ]:
model.freeze_feature_extractor()

메모리를 절약하기 위해 그라데이션 체크포인팅을 활성화

In [ ]:
model.gradient_checkpointing_enable()

## TrainingArguments

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

## Trainer

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train, # train_ds
    eval_dataset=common_voice_test, # test_ds
    tokenizer=processor.feature_extractor,
)

## Model Training

In [ ]:
trainer.train()

CTC 손실을 사용하여 더 큰 데이터 세트에서 더 큰 모델을 미세 조정하려면 [여기서](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) 공식 음성 인식 예를 살펴봐야 한다.

## Model predict

In [ ]:
model(audio_data)